# From this notebook, we can see how various data processing methods, data cleaning steps have been taken. Finally we made the train and test dataset for model  building

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
import pandas as pd
import re
import spacy
from tqdm import tqdm
import time
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Download stopwords if not already
# nltk.download('stopwords')

# Load spacy model
# nlp = spacy.load('en_core_web_sm')

# drive.mount('/content/drive')

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/train_cleaned.csv",low_memory=False)
# print(f"Number of nulls: {train['crimeaditionalinfo'].isnull().sum()}")
# train['msg'] = train['crimeaditionalinfo'].fillna("Null")
# train = train[['msg','category','sub_category']]
print(train.shape)

test = pd.read_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/test_cleaned.csv",low_memory=False)
# print(f"Number of nulls: {test['crimeaditionalinfo'].isnull().sum()}")
# test['msg'] = test['crimeaditionalinfo'].fillna("Null")
# test = test[['msg','category','sub_category']]
print(test.shape)

print(train.shape)
train = train.drop_duplicates('msg')
print(train.shape)

print(test.shape)
test = test.drop_duplicates('msg')
print(test.shape)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Mounted at /content/drive
(85014, 6)
(28786, 5)
(85014, 6)
(85014, 6)
(28786, 5)
(28786, 5)


In [ ]:
train.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_train.csv",index=False)
test.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_test.csv",index=False)

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_train.csv",low_memory=False)
test = pd.read_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_train.csv",low_memory=False)

print(train.shape, test.shape)

# Remove blank space msgs
train = train[~train['crimeaditionalinfo'].isnull()].reset_index(drop=True)
train = train[train['crimeaditionalinfo'].str.strip() != ''].reset_index(drop=True)

# Remove strings with less than 15 characters
train = train[(train['crimeaditionalinfo'].apply(len) > 15)].reset_index(drop=True)
print(train.shape, test.shape)

# Remove blank space msgs
test = test[~test['crimeaditionalinfo'].isnull()].reset_index(drop=True)
test = test[test['crimeaditionalinfo'].str.strip() != ''].reset_index(drop=True)

# Remove strings with less than 15 characters
test = test[(test['crimeaditionalinfo'].apply(len) > 15)].reset_index(drop=True)
print(train.shape, test.shape)

train = train.drop_duplicates('crimeaditionalinfo')
test = test.drop_duplicates('crimeaditionalinfo')

print(train.shape, test.shape)

train['msg'] = train['crimeaditionalinfo'].copy()
test['msg'] = test['crimeaditionalinfo'].copy()

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_train.csv",low_memory=False)
test = pd.read_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_test.csv",low_memory=False)

print(train.shape, test.shape)

(84116, 10) (28364, 10)


## Idealogy

1.   **Identify grammatically wrong words:**
  1.   Hinglish words
  *  Google Translate
  2.   Misspelled words - ungrammatical mistakes, space-less words getting combined.
  3. Handling edge cases - less character msgs, msg doesnt making sense.
  4. Removing entities to reduce randomness

2.    **Cleaning msg**
  1. Stop words removal
  2. lower all words
  3. Remove all numbers
  4. Remove all emails
  5. Stemming/Lemma
3.    **EDA**
  1. Check msg patterns by clustering
  2. Identifying patterns in low count clusters
  3. Identifying patterns in high count clusters
  4. Unigram and Bigram classifier approach
4.   **Modelling**
  1. Classification of Category
  * **Bag of Words** - on whole set vs selected clusters
  * **TF-IDF** - on whole set vs selected clusters

  2. **Classification of Sub-Category**
  * Sub-Classifiers within Category




## Required data metrics

1. category
2. sub_category
3. msg
5. final_SLL_msg
6. SRLL_msg
7. final_SRLL_msg
8. final_category

## SRULL


In [ ]:
changing_words = {'hai': 'is', 'ki': 'of', 'se': 'from', 'received': 'receive', 'ke': 'of', 'mere': 'my', 'started': 'start', 'ka': 'of',
'dont': 'do not', 'rupees': 'rs', 'ko': 'to', 'aur': 'and', 'kiya': 'did', 'mujhe': 'me', 'par': 'but', 'liye': 'for', 'ek': 'one',
'utr': 'answer', 'karta': 'does', 'maine': 'i', 'kr': 'do', 'isnt': 'is not', 'ho': 'yes', 'bhi': 'too', 'fir': 'then', 'mera': 'my', 'cant': 'can not',
'aaya': 'came', 'paise': 'money', 'bola': 'said', 'diya': 'gave', 'im': 'i am', 'nhi': 'no', 'froud': 'fraud', 'ive': 'i have', 'nahi': 'no', 'liya': 'took',
'ye': 'these', 'meri': 'my', 'plz': 'please', 'aap': 'you', 'gya': 'went', 'mene': 'i have', 'rupee': 'rs', 'hu': 'am', 'uske': 'his', 'jo': 'who', 'pls': 'please',
'aadhar': 'aadhaar', 'karne': 'to do', 'gaya': 'went', 'wasnt': 'was not', 'inr': 'rs', 'wo': 'they', 'baad': 'after', 'rahe': 'are', 'de': 'give',
'u': 'you', 'koi': 'any', 'paisa': 'rs', 'hua': 'happened', 'january': 'jan', 'sath': 'with', 'anydesk': 'any desk', 'nd': 'and', 'baroda': 'bob', 'frauder': 'fraud',
'pese': 'rs', 'thats': 'that is', 'aya': 'came', 'kuch': 'some', 'krta': 'does', 'gye': 'went', 'karke': 'by doing', 'rhe': 'are', 'unhone': 'they', 'kaha': 'said',
'na': 'no', 'bataya': 'told', 'fb': 'facebook', 'bol': 'tell', 'naam': 'names', 'krne': 'to do', 'unke': 'their', 'ab': 'now', 'phir': 'then', 'thi': 'was',
'baat': 'talk', 'log': 'people', 'ref': 'reference', 'muje': 'me', 'oneoff': 'one off', 'rupey': 'rs', 'hadnt': 'had not', 'phonepay': 'phonepe', 'din': 'day',
'everyones': 'everyone', 'relationships': 'relationship', 'diye': 'given', 'runaround': 'run around', 'kisi': 'any', 'vo': 'they', 'rupay': 'rs',
'apne': 'our', 'ltd': 'limited', 'btaya': 'told', 'le': 'take', 'uska': 'his', 'aa': 'come', 'gaye': 'went', 'frod': 'fraud', 'february': 'feb', 'bhej': 'send',
'hain': 'are', 'mein': 'in', 'dhamki': 'threat', 'rupe': 'rs', 'frouder': 'fraud', 'kare': 'do', 'bahut': 'very', 'rupaye': 'rs', 'december': 'dec',
'apna': 'own', 'jab': 'when', 'ie': 'that is', 'iam': 'i am', 'sirmadam': 'sir madam', 'lekin': 'but', 'dene': 'to give', 'messaged': 'messaged',
'abhi': 'now', 'mujhse': 'from me', 'aapko': 'to you', 'vedio': 'video', 'bat': 'talk', 'pas': 'near', 'ammount': 'amount', 'creditdebit': 'credit debit',
'ha': 'yes', 'lo': 'take', 'toh': 'so', 'karna': 'to do', 'ur': 'your', 'didnot': 'did not', 'sab': 'all', 'rahi': 'doing', 'karo': 'do it', 'gali': 'street',
'inke': 'their', 'pvt': 'private', 'holdreverse': 'hold reverse', 'jis': 'that', 'hoga': 'will be', 'kiye': 'did', 'dena': 'give', 'apko': 'to you',
'watsapp': 'whatsapp', 'krke': 'by doing', 'nikal': 'remove', 'asap': 'as soon as possible', 'karte': 'do', 'usko': 'for him', 'suspecious': 'suspicious',
'tak': 'till', 'jayega': 'will go', 'bheja': 'sent', 'peoples': 'people', 'unathorised': 'unauthorised', 'otps': 'otp', 'hackd': 'hacked', 'ap': 'you',
'jisme': 'in which', 'hrs': 'hours', 'dwara': 'by', 'laga': 'put', 'ji': 'yes', 'fruad': 'fraud', 'bta': 'tell me', 'jaye': 'go', 'aapke': 'your',
'aaj': 'today', 'pata': 'know', 'kijiye': 'do it', 'ni': 'no', 'jiska': 'whose', 'pornographic': 'pornography', 'fruader': 'fraud', 'fruadster': 'fraud',
'fruads': 'fraud', 'fruaded': 'fraud', 'gpayphone': 'gpay phone', 'phonepegpay': 'phonepe gpay', 'fruade': 'fraud', 'fruadent': 'fraud', 'paytmgpay': 'paytm gpay',
'gpayphonepay': 'gpay phonepe', 'pornograpy': 'pornography', 'fruadest': 'fraud', 'fruadsters': 'fraud', 'fruading': 'fraud', 'cimplaint': 'complaint',
'gpaypaytm': 'gpay paytm', 'gpayeuronettotal': 'gpay euronettotal', 'phonepaygpay': 'phonepe gpay', 'pornogragy': 'pornography', 'childporns': 'child porn',
'gpayat': 'gpay at', 'gpayhe': 'is gpay', 'govtsalary': 'government salary', 'gpaymy': 'gpay my', 'gpayphonepe': 'gpay phonepe', 'bygpay': 'by gpay',
'gpayed': 'gpay', 'gpaybut': 'gpay but', 'gpaythey': 'gpay they', 'afruad': 'a fraud', 'fruadulant': 'fraud', 'pornograpgy': 'pornography',
'gpaythen': 'gpay then', 'fruadulent': 'fraud', 'hellomy': 'hello my', 'bankingpay': 'banking pay', 'gpayinitial': 'gpay initial', 'gpaylater': 'gpay later',
'whatappgpay': 'whatsapp gpay', 'fruadamount': 'fraud amount', 'lostvictim': 'lost victim', 'padhega': 'will read', 'varificaon': 'verification',
'maiupulating': 'manipulating', 'nowadys': 'now a days', 'hasband': 'husband', 'subcribers': 'subscriber', 'youtubewhen': 'youtube when', 'instarction': 'interaction',
'cardupimobile': 'card upi mobile', 'rejecied': 'rejected', 'bhahen': 'sister', 'comingthey': 'coming they', 'amountfirsy': 'amount first', 'uhave': 'you have',
'roadowner': 'road owner', 'readable': 'read able', 'paasport': 'passport', 'latere': 'later', 'tryinh': 'trying', 'gpayon': 'gpay on', 'praveengpay': 'praveen gpay',
'upipaytmgpay': 'upi paytm gpay', 'gpaylink': 'gpay link', 'gpayamount': 'gpay amount', 'intrnet': "internet", "secreate": "secret", "victime": "victim",
"apka": "your", "apke": "your", 'aapka': 'yours', 'abusive': 'abuse', 'acc': 'account', 'accidentally': 'accident', 'ad': 'advertisement', 'adhar': 'aadhaar', 'alag': 'apart',
'amt': 'amount', 'apka': 'your', 'apke': 'your', 'app': 'application', 'approval': 'approve', 'attachment': 'attach', 'baar': 'times', 'band': 'close',
'bankar': 'become', 'banking': 'bank', 'bar': 'times', 'bata': 'tell', 'blackmailing': 'blackmail', 'booking': 'book', 'careful': 'care',
'chala': 'walked', 'chesaru': 'done', 'clothe': 'cloth', 'complainant': 'complaint', 'completely': 'complete', 'confirmation': 'confirm',
'constantly': 'constant', 'continuously': 'continuous', 'dal': 'did', 'deduction': 'deduct', 'delivery': 'deliver', 'denge': 'give',
'dhani': 'rich', 'di': 'gave', 'dijiye': 'give', 'directly': 'direct', 'dist': 'district', 'dunga': 'give', 'firstly': 'first', 'fraudster': 'fraud',
'fraudulent': 'fraud', 'frezzing': 'freeze', 'froder': 'fraud', 'frustrated': 'frustrate', 'galat': 'wrong', 'gandi': 'dirty', 'getting': 'get',
'googlepay': 'gpay', 'harassment': 'harass', 'harrasse': 'harass', 'holder': 'hold', 'hone': 'to', 'honestly': 'honest', 'hum': 'we',
'immediately': 'immediate', 'impersonation': 'impersonate', 'informed': 'inform', 'initially': 'initial', 'insta': 'instagram', 'instal': 'install',
'instruction': 'instruct', 'investigation': 'investigate', 'isne': 'has', 'ja': 'go', 'jankar': 'knowing', 'jankari': 'information',
'jiske': 'whose', 'kar': 'do', 'karwa': 'did', 'kat': 'cut', 'kra': 'do', 'kro': 'do', 'kya': 'what', 'lac': 'lakh', 'ladki': 'lady', 'lal': 'red',
'li': 'took', 'log': 'people', 'lone': 'loan', 'mai': 'i', 'mang': 'demand', 'mentally': 'mental', 'mila': 'found', 'min': 'minimum', 'mo': 'mobile',
'mob': 'mobile', 'msg': 'message', 'msgs': 'message', 'nam': 'name', 'ne': 'near', 'nos': 'number', 'notifiction': 'notification', 'num': 'number',
'pahle': 'first', 'pareshan': 'worried', 'pe': 'pay', 'pende': 'pending', 'ph': 'phone', 'pic': 'picture', 'pr': 'but', 'pta': 'know', 'raha': 'remained',
'recieve': 'receive', 'recive': 'receive', 'recording': 'record', 'reference': 'reference', 'refrence': 'reference', 'refunded': 'refund', 'rha': 'remained',
'rhi': 'doing', 'sare': 'ok', 'saving': 'save', 'scamme': 'scammer', 'smishe': 'funny', 'st': 'street', 'stange': 'strange', 'suchana': 'information',
'suchna': 'information', 'suchnakrta': 'inform ', 'terrifying': 'terrify', 'texte': 'text', 'thana': 'station', 'threate': 'threat', 'trans': 'transaction',
'transacation': 'transaction', 'transaction': 'transaction', 'transction': 'transaction', 'transection': 'transaction', 'usme': 'in that', 'usne': 'he',
'vishe': 'special', 'wale': 'ones', 'wapas': 'back', 'week': 'weak', 'whatsup': 'whatsapp', 'withdrawal': 'withdraw', 'working': 'work', 'ya': 'yes'
}

In [ ]:
a[a['value'].eq("fraud")]

,key,value
33,froud,fraud
66,frauder,fraud
110,frod,fraud
117,frouder,fraud
176,fruad,fraud
186,fruader,fraud
187,fruadster,fraud
188,fruads,fraud
189,fruaded,fraud
192,fruade,fraud


In [ ]:
a = pd.Series(changing_words).reset_index()
a.columns = ['key', 'value']
a['value'].value_counts()

,count
value,
fraud,19
rs,9
do,6
give,5
transaction,5
...,...
gpay phone,1
paytm gpay,1
gpay paytm,1


In [ ]:
def do_correction(text):
    changing_words = {'hai': 'is', 'ki': 'of', 'se': 'from', 'received': 'receive', 'ke': 'of', 'mere': 'my', 'started': 'start', 'ka': 'of',
    'dont': 'do not', 'rupees': 'rs', 'ko': 'to', 'aur': 'and', 'kiya': 'did', 'mujhe': 'me', 'par': 'but', 'liye': 'for', 'ek': 'one',
    'utr': 'answer', 'karta': 'does', 'maine': 'i', 'kr': 'do', 'isnt': 'is not', 'ho': 'yes', 'bhi': 'too', 'fir': 'then', 'mera': 'my', 'cant': 'can not',
    'aaya': 'came', 'paise': 'money', 'bola': 'said', 'diya': 'gave', 'im': 'i am', 'nhi': 'no', 'froud': 'fraud', 'ive': 'i have', 'nahi': 'no', 'liya': 'took',
    'ye': 'these', 'meri': 'my', 'plz': 'please', 'aap': 'you', 'gya': 'went', 'mene': 'i have', 'rupee': 'rs', 'hu': 'am', 'uske': 'his', 'jo': 'who', 'pls': 'please',
    'aadhar': 'aadhaar', 'karne': 'to do', 'gaya': 'went', 'wasnt': 'was not', 'inr': 'rs', 'wo': 'they', 'baad': 'after', 'rahe': 'are', 'de': 'give',
    'u': 'you', 'koi': 'any', 'paisa': 'rs', 'hua': 'happened', 'january': 'jan', 'sath': 'with', 'anydesk': 'any desk', 'nd': 'and', 'baroda': 'bob', 'frauder': 'fraud',
    'pese': 'rs', 'thats': 'that is', 'aya': 'came', 'kuch': 'some', 'krta': 'does', 'gye': 'went', 'karke': 'by doing', 'rhe': 'are', 'unhone': 'they', 'kaha': 'said',
    'na': 'no', 'bataya': 'told', 'fb': 'facebook', 'bol': 'tell', 'naam': 'names', 'krne': 'to do', 'unke': 'their', 'ab': 'now', 'phir': 'then', 'thi': 'was',
    'baat': 'talk', 'log': 'people', 'ref': 'reference', 'muje': 'me', 'oneoff': 'one off', 'rupey': 'rs', 'hadnt': 'had not', 'phonepay': 'phonepe', 'din': 'day',
    'everyones': 'everyone', 'relationships': 'relationship', 'diye': 'given', 'runaround': 'run around', 'kisi': 'any', 'vo': 'they', 'rupay': 'rs',
    'apne': 'our', 'ltd': 'limited', 'btaya': 'told', 'le': 'take', 'uska': 'his', 'aa': 'come', 'gaye': 'went', 'frod': 'fraud', 'february': 'feb', 'bhej': 'send',
    'hain': 'are', 'mein': 'in', 'dhamki': 'threat', 'rupe': 'rs', 'frouder': 'fraud', 'kare': 'do', 'bahut': 'very', 'rupaye': 'rs', 'december': 'dec',
    'apna': 'own', 'jab': 'when', 'ie': 'that is', 'iam': 'i am', 'sirmadam': 'sir madam', 'lekin': 'but', 'dene': 'to give', 'messaged': 'messaged',
    'abhi': 'now', 'mujhse': 'from me', 'aapko': 'to you', 'vedio': 'video', 'bat': 'talk', 'pas': 'near', 'ammount': 'amount', 'creditdebit': 'credit debit',
    'ha': 'yes', 'lo': 'take', 'toh': 'so', 'karna': 'to do', 'ur': 'your', 'didnot': 'did not', 'sab': 'all', 'rahi': 'doing', 'karo': 'do it', 'gali': 'street',
    'inke': 'their', 'pvt': 'private', 'holdreverse': 'hold reverse', 'jis': 'that', 'hoga': 'will be', 'kiye': 'did', 'dena': 'give', 'apko': 'to you',
    'watsapp': 'whatsapp', 'krke': 'by doing', 'nikal': 'remove', 'asap': 'as soon as possible', 'karte': 'do', 'usko': 'for him', 'suspecious': 'suspicious',
    'tak': 'till', 'jayega': 'will go', 'bheja': 'sent', 'peoples': 'people', 'unathorised': 'unauthorised', 'otps': 'otp', 'hackd': 'hacked', 'ap': 'you',
    'jisme': 'in which', 'hrs': 'hours', 'dwara': 'by', 'laga': 'put', 'ji': 'yes', 'fruad': 'fraud', 'bta': 'tell me', 'jaye': 'go', 'aapke': 'your',
    'aaj': 'today', 'pata': 'know', 'kijiye': 'do it', 'ni': 'no', 'jiska': 'whose', 'pornographic': 'pornography', 'fruader': 'fraud', 'fruadster': 'fraud',
    'fruads': 'fraud', 'fruaded': 'fraud', 'gpayphone': 'gpay phone', 'phonepegpay': 'phonepe gpay', 'fruade': 'fraud', 'fruadent': 'fraud', 'paytmgpay': 'paytm gpay',
    'gpayphonepay': 'gpay phonepe', 'pornograpy': 'pornography', 'fruadest': 'fraud', 'fruadsters': 'fraud', 'fruading': 'fraud', 'cimplaint': 'complaint',
    'gpaypaytm': 'gpay paytm', 'gpayeuronettotal': 'gpay euronettotal', 'phonepaygpay': 'phonepe gpay', 'pornogragy': 'pornography', 'childporns': 'child porn',
    'gpayat': 'gpay at', 'gpayhe': 'is gpay', 'govtsalary': 'government salary', 'gpaymy': 'gpay my', 'gpayphonepe': 'gpay phonepe', 'bygpay': 'by gpay',
    'gpayed': 'gpay', 'gpaybut': 'gpay but', 'gpaythey': 'gpay they', 'afruad': 'a fraud', 'fruadulant': 'fraud', 'pornograpgy': 'pornography',
    'gpaythen': 'gpay then', 'fruadulent': 'fraud', 'hellomy': 'hello my', 'bankingpay': 'banking pay', 'gpayinitial': 'gpay initial', 'gpaylater': 'gpay later',
    'whatappgpay': 'whatsapp gpay', 'fruadamount': 'fraud amount', 'lostvictim': 'lost victim', 'padhega': 'will read', 'varificaon': 'verification',
    'maiupulating': 'manipulating', 'nowadys': 'now a days', 'hasband': 'husband', 'subcribers': 'subscriber', 'youtubewhen': 'youtube when', 'instarction': 'interaction',
    'cardupimobile': 'card upi mobile', 'rejecied': 'rejected', 'bhahen': 'sister', 'comingthey': 'coming they', 'amountfirsy': 'amount first', 'uhave': 'you have',
    'roadowner': 'road owner', 'readable': 'read able', 'paasport': 'passport', 'latere': 'later', 'tryinh': 'trying', 'gpayon': 'gpay on', 'praveengpay': 'praveen gpay',
    'upipaytmgpay': 'upi paytm gpay', 'gpaylink': 'gpay link', 'gpayamount': 'gpay amount', 'intrnet': "internet", "secreate": "secret", "victime": "victim",
    "apka": "your", "apke": "your", 'aapka': 'yours', 'abusive': 'abuse', 'acc': 'account', 'accidentally': 'accident', 'ad': 'advertisement', 'adhar': 'aadhaar', 'alag': 'apart',
    'amt': 'amount', 'apka': 'your', 'apke': 'your', 'app': 'application', 'approval': 'approve', 'attachment': 'attach', 'baar': 'times', 'band': 'close',
    'bankar': 'become', 'banking': 'bank', 'bar': 'times', 'bata': 'tell', 'blackmailing': 'blackmail', 'booking': 'book', 'careful': 'care',
    'chala': 'walked', 'chesaru': 'done', 'clothe': 'cloth', 'complainant': 'complaint', 'completely': 'complete', 'confirmation': 'confirm',
    'constantly': 'constant', 'continuously': 'continuous', 'dal': 'did', 'deduction': 'deduct', 'delivery': 'deliver', 'denge': 'give',
    'dhani': 'rich', 'di': 'gave', 'dijiye': 'give', 'directly': 'direct', 'dist': 'district', 'dunga': 'give', 'firstly': 'first', 'fraudster': 'fraud',
    'fraudulent': 'fraud', 'frezzing': 'freeze', 'froder': 'fraud', 'frustrated': 'frustrate', 'galat': 'wrong', 'gandi': 'dirty', 'getting': 'get',
    'googlepay': 'gpay', 'harassment': 'harass', 'harrasse': 'harass', 'holder': 'hold', 'hone': 'to', 'honestly': 'honest', 'hum': 'we',
    'immediately': 'immediate', 'impersonation': 'impersonate', 'informed': 'inform', 'initially': 'initial', 'insta': 'instagram', 'instal': 'install',
    'instruction': 'instruct', 'investigation': 'investigate', 'isne': 'has', 'ja': 'go', 'jankar': 'knowing', 'jankari': 'information',
    'jiske': 'whose', 'kar': 'do', 'karwa': 'did', 'kat': 'cut', 'kra': 'do', 'kro': 'do', 'kya': 'what', 'lac': 'lakh', 'ladki': 'lady', 'lal': 'red',
    'li': 'took', 'log': 'people', 'lone': 'loan', 'mai': 'i', 'mang': 'demand', 'mentally': 'mental', 'mila': 'found', 'min': 'minimum', 'mo': 'mobile',
    'mob': 'mobile', 'msg': 'message', 'msgs': 'message', 'nam': 'name', 'ne': 'near', 'nos': 'number', 'notifiction': 'notification', 'num': 'number',
    'pahle': 'first', 'pareshan': 'worried', 'pe': 'pay', 'pende': 'pending', 'ph': 'phone', 'pic': 'picture', 'pr': 'but', 'pta': 'know', 'raha': 'remained',
    'recieve': 'receive', 'recive': 'receive', 'recording': 'record', 'reference': 'reference', 'refrence': 'reference', 'refunded': 'refund', 'rha': 'remained',
    'rhi': 'doing', 'sare': 'ok', 'saving': 'save', 'scamme': 'scammer', 'smishe': 'funny', 'st': 'street', 'stange': 'strange', 'suchana': 'information',
    'suchna': 'information', 'suchnakrta': 'inform ', 'terrifying': 'terrify', 'texte': 'text', 'thana': 'station', 'threate': 'threat', 'trans': 'transaction',
    'transacation': 'transaction', 'transaction': 'transaction', 'transction': 'transaction', 'transection': 'transaction', 'usme': 'in that', 'usne': 'he',
    'vishe': 'special', 'wale': 'ones', 'wapas': 'back', 'week': 'weak', 'whatsup': 'whatsapp', 'withdrawal': 'withdraw', 'working': 'work', 'ya': 'yes'
    }

    # Correct the words
    corrected_msg = ''
    all_keys = changing_words.keys()
    for word in text.split():
        if word in all_keys:
            corrected_msg += changing_words[word]
        else:
            corrected_msg += word
        corrected_msg += " "
    return corrected_msg

def do_sensitive_website_replacement(corrected_msg, website_match_findings):
  sensitive_keywords = [
        "porn", "xxx", "hentai", "xvideo", "adult", "breast","pussy", "dick", "cock","sex", "erotic", "nude", "camgirl", "cams", "cheat",
        "escort", "fetish", "bdsm", "taboo", "incest", "shemale", "milf", "barelylegal", "amateur",
        "anal", "fetish", "hotgirls", "kinky", "hardcore", "lingerie", "webcam", "strip", "nsfw",
        "onlyfans", "redtube", "youjizz", "spankbang", "tnaflix", "xtube", "nudity", "playboy",
        "bangbros", "brazzers", "naughty", "mature", "teens", "voyeur", "threesome", "gangbang",
        "peeping", "exposed", "babes", "slut", "masturbate", "hookup", "sugarbaby", "paypig", "comic"
  ]

  new_msg = ''
  for word in corrected_msg.strip().split():
    # print(word)
    if len(website_match_findings) > 0:
        if word in website_match_findings and 'express' not in word and 'gmail' not in word and 'internet' not in word:
            # Check if it has sensitive word
            for sen_word in sensitive_keywords:
                if sen_word in word:
                    new_msg += sen_word
        else:
          new_msg += word
    else:
        new_msg += word
    new_msg += " "
  return new_msg

In [ ]:
# final SLL msg
from tqdm import tqdm
## DATA PROCESSING
'''
1. stopwords
2. lower
3. replace mispelled words
4. detect websites and detect sensitive words and replace them, else remove
5. lemma
'''
stop_words = [
    "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according",
    "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag",
    "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although",
    "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone",
    "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar",
    "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available",
    "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes",
    "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside",
    "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief",
    "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause",
    "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co",
    "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding",
    "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d",
    "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different",
    "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due",
    "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el",
    "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et",
    "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey",
    "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo",
    "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full",
    "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes",
    "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt",
    "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein",
    "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully",
    "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id",
    "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch",
    "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward",
    "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr",
    "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last",
    "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely",
    "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make",
    "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million",
    "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't",
    "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't",
    "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless",
    "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained",
    "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only",
    "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over",
    "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas",
    "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible",
    "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud",
    "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re",
    "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research",
    "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt",
    "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section",
    "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf",
    "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown",
    "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm",
    "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp",
    "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such",
    "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten",
    "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves",
    "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto",
    "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this",
    "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til",
    "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying",
    "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately",
    "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses",
    "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs",
    "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went",
    "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's",
    "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither",
    "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish",
    "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't",
    "xt", "xv", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself",
    "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz", 'ag', 'amit', 'ani', 'dec', 'delhi', 'devi', 'feb', 'five', 'gupta', 'hi', 'hui',
    'jaipur', 'jan', 'kumar', 'may', 'rahul', 'ram', 'sharma', 'singh', 'yadav', 'mar', 'apr', 'jun', 'jul', 'aug', 'sept', 'oct', 'nov']

train_SRULL = []
train_SLL = []
train_SRLL = []

for msg in tqdm(train['msg'], desc = 'processing train:'):
  # SLL
  text = msg.lower().strip()
  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words]
  # Lemmatization using spacy
  doc = nlp(' '.join(tokens))
  lemmatized_tokens = [token.lemma_ for token in doc]
  train_SLL.append(' '.join(lemmatized_tokens).strip())

  # SRLL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)
  tokens = corrected_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  lemmatized_tokens = [token.lemma_ for token in doc]
  train_SRLL.append(' '.join(lemmatized_tokens).strip())

  # SRULL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)

  doubt_website_pattern = r'\b(?:http|https|www)[a-zA-Z0-9]+(?:dot|at)?[a-zA-Z0-9]+(?:com|org|net)\b|\b[a-zA-Z0-9._%+-]+(?:at|dot)?[a-zA-Z0-9]+(?:com|org|net)\b'
  website_match_findings = re.findall(doubt_website_pattern, corrected_msg)
  new_msg = do_sensitive_website_replacement(corrected_msg, website_match_findings)

  tokens = new_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  lemmatized_tokens = [token.lemma_ for token in doc]
  train_SRULL.append(' '.join(lemmatized_tokens).strip())

train['SRULL_msg_updated'] = train_SRULL
train['SLL_msg_updated'] = train_SLL
train['SRLL_msg_updated'] = train_SRLL

train.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_train_updated_2_11_2024.csv",index=False)
print("Train saved")

test_SLL = []
test_SRLL = []
test_SRULL = []

for msg in tqdm(test['msg'], desc = 'processing train:'):
  # SLL
  text = msg.lower().strip()
  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words]
  # Lemmatization using spacy
  doc = nlp(' '.join(tokens))
  lemmatized_tokens = [token.lemma_ for token in doc]
  test_SLL.append(' '.join(lemmatized_tokens).strip())

  # SRLL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)
  tokens = corrected_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  lemmatized_tokens = [token.lemma_ for token in doc]
  test_SRLL.append(' '.join(lemmatized_tokens).strip())

  # SRULL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)

  doubt_website_pattern = r'\b(?:http|https|www)[a-zA-Z0-9]+(?:dot|at)?[a-zA-Z0-9]+(?:com|org|net)\b|\b[a-zA-Z0-9._%+-]+(?:at|dot)?[a-zA-Z0-9]+(?:com|org|net)\b'
  website_match_findings = re.findall(doubt_website_pattern, corrected_msg)
  new_msg = do_sensitive_website_replacement(corrected_msg, website_match_findings)

  tokens = new_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  lemmatized_tokens = [token.lemma_ for token in doc]
  test_SRULL.append(' '.join(lemmatized_tokens).strip())

test['SLL_msg_updated'] = test_SLL
test['SRLL_msg_updated'] = test_SRLL
test['SRULL_msg_updated'] = test_SRULL

test.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_test_updated_2_11_2024.csv",index=False)

processing train:: 100%|██████████| 84116/84116 [21:41<00:00, 64.63it/s]


Train saved


processing train:: 100%|██████████| 28364/28364 [07:21<00:00, 64.18it/s]


In [ ]:
empty_words = test[test['SRULL_msg'].eq("")]

empty = []
for msg in tqdm(empty_words['msg'], desc = 'processing empty words:'):
  text = msg.lower().strip()

  # Manual correction
  corrected_msg = do_correction(text)

  # Detect all possible combinations of website search
  doubt_website_pattern = r'\b(?:http|https|www)[a-zA-Z0-9]+(?:dot|at)?[a-zA-Z0-9]+(?:com|org|net)\b|\b[a-zA-Z0-9._%+-]+(?:at|dot)?[a-zA-Z0-9]+(?:com|org|net)\b'
  # website_pattern = r'\b(?:http|https|www)\S*(?:com|org|net|edu|gov|io|co)\b'
  website_match_findings = re.findall(doubt_website_pattern, corrected_msg)

  # Get sensitive websites replaced
  new_msg = do_sensitive_website_replacement(corrected_msg, website_match_findings)

  # Tokenization and remove stopwords
  tokens = new_msg.split()
  tokens = [word for word in tokens if word not in stop_words]

  # Lemmatization using spacy
  doc = nlp(' '.join(tokens))
  lemmatized_tokens = [token.lemma_ for token in doc]

  new_msg_final = ' '.join(lemmatized_tokens)
  empty.append(new_msg_final)

processing empty words:: 100%|██████████| 2625/2625 [00:51<00:00, 51.22it/s]


## SLL

In [ ]:
# final SLL msg
from tqdm import tqdm
## DATA PROCESSING
'''
1. stopwords
2. lemma
3. lower
'''
stop_words = set(stopwords.words('english'))

train['before_SLL_msg'] = train['msg'].str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', x))
test['before_SLL_msg'] = test['msg'].str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', x))

train_SLL = []
for msg in tqdm(train['before_SLL_msg'], desc = 'processing train:'):
  text = msg
  # Tokenization and remove stopwords
  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words]

  # Lemmatization using spacy
  doc = nlp(' '.join(tokens))
  lemmatized_tokens = [token.lemma_ for token in doc]

  new_msg = ' '.join(lemmatized_tokens)
  train_SLL.append(new_msg)

train['SLL_msg'] = train_SLL

test_SLL = []
for msg in tqdm(test['before_SLL_msg'], desc = 'processing test:'):
  text = msg
  # Tokenization and remove stopwords
  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words]

  # Lemmatization using spacy
  doc = nlp(' '.join(tokens))
  lemmatized_tokens = [token.lemma_ for token in doc]

  new_msg = ' '.join(lemmatized_tokens)
  test_SLL.append(new_msg)

test['SLL_msg'] = test_SLL

train.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_train.csv",index=False)
test.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_test.csv",index=False)

processing test:: 100%|██████████| 28364/28364 [07:25<00:00, 63.67it/s]


In [ ]:
# @title
changing_words = {'hai': 'is', 'ki': 'of', 'se': 'from', 'received': 'receive', 'ke': 'of', 'mere': 'my', 'started': 'start', 'ka': 'of',
 'dont': 'do not', 'rupees': 'rs', 'ko': 'to', 'aur': 'and', 'kiya': 'did', 'mujhe': 'me', 'par': 'but', 'liye': 'for', 'ek': 'one',
 'utr': 'answer', 'karta': 'does', 'maine': 'i', 'kr': 'do', 'isnt': 'is not', 'ho': 'yes', 'bhi': 'too', 'fir': 'then', 'mera': 'my', 'cant': 'can not',
 'aaya': 'came', 'paise': 'money', 'bola': 'said', 'diya': 'gave', 'im': 'i am', 'nhi': 'no', 'froud': 'fraud', 'ive': 'i have', 'nahi': 'no', 'liya': 'took',
 'ye': 'these', 'meri': 'my', 'plz': 'please', 'aap': 'you', 'gya': 'went', 'mene': 'i have', 'rupee': 'rs', 'hu': 'am', 'uske': 'his', 'jo': 'who', 'pls': 'please',
 'aadhar': 'aadhaar', 'karne': 'to do', 'gaya': 'went', 'wasnt': 'was not', 'inr': 'rs', 'wo': 'they', 'baad': 'after', 'rahe': 'are', 'de': 'give',
 'u': 'you', 'koi': 'any', 'paisa': 'rs', 'hua': 'happened', 'january': 'jan', 'sath': 'with', 'anydesk': 'any desk', 'nd': 'and', 'baroda': 'bob', 'frauder': 'fraud',
 'pese': 'rs', 'thats': 'that is', 'aya': 'came', 'kuch': 'some', 'krta': 'does', 'gye': 'went', 'karke': 'by doing', 'rhe': 'are', 'unhone': 'they', 'kaha': 'said',
 'na': 'no', 'bataya': 'told', 'fb': 'facebook', 'bol': 'tell', 'naam': 'names', 'krne': 'to do', 'unke': 'their', 'ab': 'now', 'phir': 'then', 'thi': 'was',
 'baat': 'talk', 'log': 'people', 'ref': 'reference', 'muje': 'me', 'oneoff': 'one off', 'rupey': 'rs', 'hadnt': 'had not', 'phonepay': 'phonepe', 'din': 'day',
 'everyones': 'everyone', 'relationships': 'relationship', 'diye': 'given', 'runaround': 'run around', 'kisi': 'any', 'vo': 'they', 'rupay': 'rs',
 'apne': 'our', 'ltd': 'limited', 'btaya': 'told', 'le': 'take', 'uska': 'his', 'aa': 'come', 'gaye': 'went', 'frod': 'fraud', 'february': 'feb', 'bhej': 'send',
 'hain': 'are', 'mein': 'in', 'dhamki': 'threat', 'rupe': 'rs', 'frouder': 'fraud', 'kare': 'do', 'bahut': 'very', 'rupaye': 'rs', 'december': 'dec',
 'apna': 'own', 'jab': 'when', 'ie': 'that is', 'iam': 'i am', 'sirmadam': 'sir madam', 'lekin': 'but', 'dene': 'to give', 'messaged': 'messaged',
 'abhi': 'now', 'mujhse': 'from me', 'aapko': 'to you', 'vedio': 'video', 'bat': 'talk', 'pas': 'near', 'ammount': 'amount', 'creditdebit': 'credit debit',
 'ha': 'yes', 'lo': 'take', 'toh': 'so', 'karna': 'to do', 'ur': 'your', 'didnot': 'did not', 'sab': 'all', 'rahi': 'doing', 'karo': 'do it', 'gali': 'street',
 'inke': 'their', 'pvt': 'private', 'holdreverse': 'hold reverse', 'jis': 'that', 'hoga': 'will be', 'kiye': 'did', 'dena': 'give', 'apko': 'to you',
 'watsapp': 'whatsapp', 'krke': 'by doing', 'nikal': 'remove', 'asap': 'as soon as possible', 'karte': 'do', 'usko': 'for him', 'suspecious': 'suspicious',
 'tak': 'till', 'jayega': 'will go', 'bheja': 'sent', 'peoples': 'people', 'unathorised': 'unauthorised', 'otps': 'otp', 'hackd': 'hacked', 'ap': 'you',
 'jisme': 'in which', 'hrs': 'hours', 'dwara': 'by', 'laga': 'put', 'ji': 'yes', 'fruad': 'fraud', 'bta': 'tell me', 'jaye': 'go', 'aapke': 'your',
 'aaj': 'today', 'pata': 'know', 'kijiye': 'do it', 'ni': 'no', 'jiska': 'whose', 'pornographic': 'pornography', 'fruader': 'fraud', 'fruadster': 'fraud',
 'fruads': 'fraud', 'fruaded': 'fraud', 'gpayphone': 'gpay phone', 'phonepegpay': 'phonepe gpay', 'fruade': 'fraud', 'fruadent': 'fraud', 'paytmgpay': 'paytm gpay',
 'gpayphonepay': 'gpay phonepe', 'pornograpy': 'pornography', 'fruadest': 'fraud', 'fruadsters': 'fraud', 'fruading': 'fraud', 'cimplaint': 'complaint',
 'gpaypaytm': 'gpay paytm', 'gpayeuronettotal': 'gpay euronettotal', 'phonepaygpay': 'phonepe gpay', 'pornogragy': 'pornography', 'childporns': 'child porn',
 'gpayat': 'gpay at', 'gpayhe': 'is gpay', 'govtsalary': 'government salary', 'gpaymy': 'gpay my', 'gpayphonepe': 'gpay phonepe', 'bygpay': 'by gpay',
 'gpayed': 'gpay', 'gpaybut': 'gpay but', 'gpaythey': 'gpay they', 'afruad': 'a fraud', 'fruadulant': 'fraud', 'pornograpgy': 'pornography',
 'gpaythen': 'gpay then', 'fruadulent': 'fraud', 'hellomy': 'hello my', 'bankingpay': 'banking pay', 'gpayinitial': 'gpay initial', 'gpaylater': 'gpay later',
 'whatappgpay': 'whatsapp gpay', 'fruadamount': 'fraud amount', 'lostvictim': 'lost victim', 'padhega': 'will read', 'varificaon': 'verification',
 'maiupulating': 'manipulating', 'nowadys': 'now a days', 'hasband': 'husband', 'subcribers': 'subscriber', 'youtubewhen': 'youtube when', 'instarction': 'interaction',
 'cardupimobile': 'card upi mobile', 'rejecied': 'rejected', 'bhahen': 'sister', 'comingthey': 'coming they', 'amountfirsy': 'amount first', 'uhave': 'you have',
 'roadowner': 'road owner', 'readable': 'read able', 'paasport': 'passport', 'latere': 'later', 'tryinh': 'trying', 'gpayon': 'gpay on', 'praveengpay': 'praveen gpay',
 'upipaytmgpay': 'upi paytm gpay', 'gpaylink': 'gpay link', 'gpayamount': 'gpay amount'}

In [ ]:
remove_re_and_replace_words(train.copy())

processing::   0%|          | 0/84116 [00:00<?, ?it/s]


['i had continue receive random calls and abusive messages in my whatsapp someone added my number in a unknown facebook group name with only girls and still getting calls from unknown numbers please help me and sort out the issue as soon as possible thank you']

In [ ]:
# final SRLL msg

## DATA PROCESSING
'''
1. stopwords
2. lemma
3. lower
'''
stop_words = set(stopwords.words('english'))

def remove_re_and_replace_words(df):
  df['SRLL_msg_before'] = df['msg'].str.lower().apply(lambda x: re.sub(r'[^a-z\s]', '', x))

  new_msg_list = []
  keys_words = changing_words.keys()

  for msg in tqdm(df['SRLL_msg_before'].tolist(), desc='processing:'):
    new_msg = ""
    for word in msg.split():
      if word in keys_words:
        new_msg += changing_words[word] + " "
      else:
        new_msg += word + " "
    new_msg_list.append(new_msg.strip())
  return new_msg_list

train['SRLL_msg_before'] = remove_re_and_replace_words(train.copy())
test['SRLL_msg_before'] = remove_re_and_replace_words(test.copy())

train.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_train.csv",index=False)
test.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_test.csv",index=False)
print("msg before done")

def lemma(df):
  SRLL = []
  for msg in tqdm(df['SRLL_msg_before'].tolist(),desc='processing:'):
    text = msg
    # Tokenization and remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization using spacy
    doc = nlp(' '.join(tokens))
    lemmatized_tokens = [token.lemma_ for token in doc]

    new_msg = ' '.join(lemmatized_tokens)
    SRLL.append(new_msg)
  return SRLL

train['SRLL_msg'] = lemma(train.copy())
test['SRLL_msg'] = lemma(test.copy())

train.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_train.csv",index=False)
test.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/all_data_transformations_test.csv",index=False)

processing:: 100%|██████████| 28364/28364 [00:00<00:00, 43439.20it/s]


msg before done


processing:: 100%|██████████| 28364/28364 [07:04<00:00, 66.89it/s]


In [ ]:
train

,category,sub_category,msg,SLL_msg,clean_msg_before_srll,SRLL_msg,final_category,before_SLL_msg,SRLL_msg_before
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,I had continue received random calls and abusi...,continue receive random call abusive message w...,continue received random calls abusive message...,continue receive random call abusive message w...,2,i had continue received random calls and abusi...,i had continue receive random calls and abusiv...
1,Online Financial Fraud,Fraud CallVishing,The above fraudster is continuously messaging ...,fraudster continuously message ask pay money s...,fraudster continuously messaging asking pay mo...,fraudster continuously message ask pay money s...,1,the above fraudster is continuously messaging ...,the above fraudster is continuously messaging ...
2,Online Gambling Betting,Online Gambling Betting,He is acting like a police and demanding for m...,act like police demand money add section text ...,acting like police demanding money adding sect...,act like police demand money add section text ...,1,he is acting like a police and demanding for m...,he is acting like a police and demanding for m...
3,Online and Social Media Related Crime,Online Job Fraud,In apna Job I have applied for job interview f...,apna job apply job interview telecalle resourc...,apna job applied job interview telecalling res...,job apply job interview telecalle resource man...,2,in apna job i have applied for job interview f...,in own job i have applied for job interview fo...
4,Online Financial Fraud,Fraud CallVishing,I received a call from lady stating that she w...,receive call lady state send new phone vivo re...,received call lady stating send new phone vivo...,receive call lady state send new phone vivo re...,1,i received a call from lady stating that she w...,i receive a call from lady stating that she wi...
...,...,...,...,...,...,...,...,...,...
84111,Online Financial Fraud,Internet Banking Related Fraud,Identity theft Smishing SMS Fraud CreditDeb...,identity theft smishe sms fraud creditdebit ca...,identity theft smishing sms fraud creditdebit ...,identity theft smishe sms fraud credit debit c...,1,identity theft smishing sms fraud creditdeb...,identity theft smishing sms fraud credit debit...
84112,Online Financial Fraud,EWallet Related Fraud,RECEIVED CALL FROM NUMBER ASKING ABOUT phone ...,receive call number ask phone pay cash back of...,received call number asking phone pay cash bac...,receive call number ask phone pay cash back of...,1,received call from number asking about phone ...,receive call from number asking about phone pa...
84113,Online Financial Fraud,UPI Related Frauds,Cyber Stalking Blackmailing PhoneSMSVOIP C...,cyber stalk blackmailing phonesmsvoip call vic...,cyber stalking blackmailing phonesmsvoip calls...,cyber stalk blackmailing phonesmsvoip call vic...,1,cyber stalking blackmailing phonesmsvoip c...,cyber stalking blackmailing phonesmsvoip calls...
84114,Online and Social Media Related Crime,Online Matrimonial Fraud,Call karke bola ki aapka lotary laga ha aru AC...,call karke bola ki aapka lotary laga ha aru ac...,call karke bola ki aapka lotary laga ha aru ac...,call say aapka lotary put yes aru ac maga aru ...,2,call karke bola ki aapka lotary laga ha aru ac...,call by doing said of aapka lotary put yes aru...


## Base line model

In [ ]:
## DATA PROCESSING
'''
1. stopwords
2. lemma
3. lower
'''
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenization and remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization using spacy
    doc = nlp(' '.join(tokens))
    lemmatized_tokens = [token.lemma_ for token in doc]

    return ' '.join(tokens)

train['SLL_msg'] = train['crimeaditionalinfo'].apply(lambda x: clean_text(x))
test['SLL_msg'] = test['crimeaditionalinfo'].apply(lambda x: clean_text(x))

In [ ]:
## Cleaning unnecessary words
# Define stopwords
stop_words = set(stopwords.words('english'))

train = train[~train['clean_msg_before_srll'].isnull()].reset_index(drop=True)
test = test[~test['clean_msg_before_srll'].isnull()].reset_index(drop=True)

def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenization and remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

s = time.time()
keys_words = changing_words.keys()

train['clean_msg_before_srll'] = train['crimeaditionalinfo'].apply(lambda x: clean_text(x))
test['clean_msg_before_srll'] = test['crimeaditionalinfo'].apply(lambda x: clean_text(x))

new_msg_train = []
for msg in train['clean_msg_before_srll']:
  new_msg = ""
  for word in msg:
    if word in keys_words:
      new_msg += changing_words[word] + " "
    else:
      new_msg += word + " "
  new_msg_train.append(new_msg)

print('train done')

new_msg_test = []
for msg in test['clean_msg_before_srll']:
  new_msg = ""
  for word in msg.split():
    if word in keys_words:
      new_msg += changing_words[word] + " "
    else:
      new_msg += word + " "
  new_msg_test.append(new_msg)

train['SRLL_msg'] = new_msg_train
test['SRLL_msg'] = new_msg_test

print('test done')
e = time.time()
print(f"Time taken: {(e-s)/60} mins")

train done
test done
Time taken: 0.2759413639704386 mins


In [ ]:
train.shape, test.shape

((84135, 6), (28369, 6))

In [ ]:
train

,category,sub_category,crimeaditionalinfo,SLL_msg,clean_msg_before_srll,SRLL_msg
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,I had continue received random calls and abusi...,continue received random calls abusive message...,continue received random calls abusive message...,continue receive random calls abusive messages...
1,Online Financial Fraud,Fraud CallVishing,The above fraudster is continuously messaging ...,fraudster continuously messaging asking pay mo...,fraudster continuously messaging asking pay mo...,fraudster continuously messaging asking pay mo...
2,Online Gambling Betting,Online Gambling Betting,He is acting like a police and demanding for m...,acting like police demanding money adding sect...,acting like police demanding money adding sect...,acting like police demanding money adding sect...
3,Online and Social Media Related Crime,Online Job Fraud,In apna Job I have applied for job interview f...,apna job applied job interview telecalling res...,apna job applied job interview telecalling res...,own job applied job interview telecalling reso...
4,Online Financial Fraud,Fraud CallVishing,I received a call from lady stating that she w...,received call lady stating send new phone vivo...,received call lady stating send new phone vivo...,receive call lady stating send new phone vivo ...
...,...,...,...,...,...,...
84111,Online Financial Fraud,Internet Banking Related Fraud,Identity theft Smishing SMS Fraud CreditDeb...,identity theft smishing sms fraud creditdebit ...,identity theft smishing sms fraud creditdebit ...,identity theft smishing sms fraud credit debit...
84112,Online Financial Fraud,EWallet Related Fraud,RECEIVED CALL FROM NUMBER ASKING ABOUT phone ...,received call number asking phone pay cash bac...,received call number asking phone pay cash bac...,receive call number asking phone pay cash back...
84113,Online Financial Fraud,UPI Related Frauds,Cyber Stalking Blackmailing PhoneSMSVOIP C...,cyber stalking blackmailing phonesmsvoip calls...,cyber stalking blackmailing phonesmsvoip calls...,cyber stalking blackmailing phonesmsvoip calls...
84114,Online and Social Media Related Crime,Online Matrimonial Fraud,Call karke bola ki aapka lotary laga ha aru AC...,call karke bola ki aapka lotary laga ha aru ac...,call karke bola ki aapka lotary laga ha aru ac...,call by doing said of aapka lotary put yes aru...


## Change some words to corrected ones including hinglish

In [ ]:
changing_words = {'hai': 'is', 'ki': 'of', 'se': 'from', 'received': 'receive', 'ke': 'of', 'mere': 'my', 'started': 'start', 'ka': 'of',
 'dont': 'do not', 'rupees': 'rs', 'ko': 'to', 'aur': 'and', 'kiya': 'did', 'mujhe': 'me', 'par': 'but', 'liye': 'for', 'ek': 'one',
 'utr': 'answer', 'karta': 'does', 'maine': 'i', 'kr': 'do', 'isnt': 'is not', 'ho': 'yes', 'bhi': 'too', 'fir': 'then', 'mera': 'my', 'cant': 'can not',
 'aaya': 'came', 'paise': 'money', 'bola': 'said', 'diya': 'gave', 'im': 'i am', 'nhi': 'no', 'froud': 'fraud', 'ive': 'i have', 'nahi': 'no', 'liya': 'took',
 'ye': 'these', 'meri': 'my', 'plz': 'please', 'aap': 'you', 'gya': 'went', 'mene': 'i have', 'rupee': 'rs', 'hu': 'am', 'uske': 'his', 'jo': 'who', 'pls': 'please',
 'aadhar': 'aadhaar', 'karne': 'to do', 'gaya': 'went', 'wasnt': 'was not', 'inr': 'rs', 'wo': 'they', 'baad': 'after', 'rahe': 'are', 'de': 'give',
 'u': 'you', 'koi': 'any', 'paisa': 'rs', 'hua': 'happened', 'january': 'jan', 'sath': 'with', 'anydesk': 'any desk', 'nd': 'and', 'baroda': 'bob', 'frauder': 'fraud',
 'pese': 'rs', 'thats': 'that is', 'aya': 'came', 'kuch': 'some', 'krta': 'does', 'gye': 'went', 'karke': 'by doing', 'rhe': 'are', 'unhone': 'they', 'kaha': 'said',
 'na': 'no', 'bataya': 'told', 'fb': 'facebook', 'bol': 'tell', 'naam': 'names', 'krne': 'to do', 'unke': 'their', 'ab': 'now', 'phir': 'then', 'thi': 'was',
 'baat': 'talk', 'log': 'people', 'ref': 'reference', 'muje': 'me', 'oneoff': 'one off', 'rupey': 'rs', 'hadnt': 'had not', 'phonepay': 'phonepe', 'din': 'day',
 'everyones': 'everyone', 'relationships': 'relationship', 'diye': 'given', 'runaround': 'run around', 'kisi': 'any', 'vo': 'they', 'rupay': 'rs',
 'apne': 'our', 'ltd': 'limited', 'btaya': 'told', 'le': 'take', 'uska': 'his', 'aa': 'come', 'gaye': 'went', 'frod': 'fraud', 'february': 'feb', 'bhej': 'send',
 'hain': 'are', 'mein': 'in', 'dhamki': 'threat', 'rupe': 'rs', 'frouder': 'fraud', 'kare': 'do', 'bahut': 'very', 'rupaye': 'rs', 'december': 'dec',
 'apna': 'own', 'jab': 'when', 'ie': 'that is', 'iam': 'i am', 'sirmadam': 'sir madam', 'lekin': 'but', 'dene': 'to give', 'messaged': 'messaged',
 'abhi': 'now', 'mujhse': 'from me', 'aapko': 'to you', 'vedio': 'video', 'bat': 'talk', 'pas': 'near', 'ammount': 'amount', 'creditdebit': 'credit debit',
 'ha': 'yes', 'lo': 'take', 'toh': 'so', 'karna': 'to do', 'ur': 'your', 'didnot': 'did not', 'sab': 'all', 'rahi': 'doing', 'karo': 'do it', 'gali': 'street',
 'inke': 'their', 'pvt': 'private', 'holdreverse': 'hold reverse', 'jis': 'that', 'hoga': 'will be', 'kiye': 'did', 'dena': 'give', 'apko': 'to you',
 'watsapp': 'whatsapp', 'krke': 'by doing', 'nikal': 'remove', 'asap': 'as soon as possible', 'karte': 'do', 'usko': 'for him', 'suspecious': 'suspicious',
 'tak': 'till', 'jayega': 'will go', 'bheja': 'sent', 'peoples': 'people', 'unathorised': 'unauthorised', 'otps': 'otp', 'hackd': 'hacked', 'ap': 'you',
 'jisme': 'in which', 'hrs': 'hours', 'dwara': 'by', 'laga': 'put', 'ji': 'yes', 'fruad': 'fraud', 'bta': 'tell me', 'jaye': 'go', 'aapke': 'your',
 'aaj': 'today', 'pata': 'know', 'kijiye': 'do it', 'ni': 'no', 'jiska': 'whose', 'pornographic': 'pornography', 'fruader': 'fraud', 'fruadster': 'fraud',
 'fruads': 'fraud', 'fruaded': 'fraud', 'gpayphone': 'gpay phone', 'phonepegpay': 'phonepe gpay', 'fruade': 'fraud', 'fruadent': 'fraud', 'paytmgpay': 'paytm gpay',
 'gpayphonepay': 'gpay phonepe', 'pornograpy': 'pornography', 'fruadest': 'fraud', 'fruadsters': 'fraud', 'fruading': 'fraud', 'cimplaint': 'complaint',
 'gpaypaytm': 'gpay paytm', 'gpayeuronettotal': 'gpay euronettotal', 'phonepaygpay': 'phonepe gpay', 'pornogragy': 'pornography', 'childporns': 'child porn',
 'gpayat': 'gpay at', 'gpayhe': 'is gpay', 'govtsalary': 'government salary', 'gpaymy': 'gpay my', 'gpayphonepe': 'gpay phonepe', 'bygpay': 'by gpay',
 'gpayed': 'gpay', 'gpaybut': 'gpay but', 'gpaythey': 'gpay they', 'afruad': 'a fraud', 'fruadulant': 'fraud', 'pornograpgy': 'pornography',
 'gpaythen': 'gpay then', 'fruadulent': 'fraud', 'hellomy': 'hello my', 'bankingpay': 'banking pay', 'gpayinitial': 'gpay initial', 'gpaylater': 'gpay later',
 'whatappgpay': 'whatsapp gpay', 'fruadamount': 'fraud amount', 'lostvictim': 'lost victim', 'padhega': 'will read', 'varificaon': 'verification',
 'maiupulating': 'manipulating', 'nowadys': 'now a days', 'hasband': 'husband', 'subcribers': 'subscriber', 'youtubewhen': 'youtube when', 'instarction': 'interaction',
 'cardupimobile': 'card upi mobile', 'rejecied': 'rejected', 'bhahen': 'sister', 'comingthey': 'coming they', 'amountfirsy': 'amount first', 'uhave': 'you have',
 'roadowner': 'road owner', 'readable': 'read able', 'paasport': 'passport', 'latere': 'later', 'tryinh': 'trying', 'gpayon': 'gpay on', 'praveengpay': 'praveen gpay',
 'upipaytmgpay': 'upi paytm gpay', 'gpaylink': 'gpay link', 'gpayamount': 'gpay amount'}

## Remove months, locations

## Identify Grammatically Incorrect Words

In [ ]:
## Cleaning unnecessary words
# Define stopwords
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenization and remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization using spacy
    doc = nlp(' '.join(tokens))
    lemmatized_tokens = [token.lemma_ for token in doc]
    # lemmatized_tokens = [token for token in doc]

    return ' '.join(tokens)

s = time.time()
keys_words = changing_words.keys()

train['clean_msg'] = train['msg'].apply(lambda x: clean_text(x))

for msg in train['clean_msg']:
  new_msg = ""
  for word in msg:
    if word in keys_words:
      new_msg += changing_words[word] + " "
    else:
      new_msg += word + " "

print('train done')
test['clean_msg'] = test['msg'].apply(lambda x: clean_text(str(x)))

for msg in test['clean_msg']:
  new_msg = ""
  for word in msg:
    if word in keys_words:
      new_msg += changing_words[word] + " "
    else:
      new_msg += word + " "

print('test done')
e = time.time()
print(f"Time taken: {(e-s)/60} mins")

train done
test done
Time taken: 29.413054672876992 mins


In [ ]:
unique_words = dict()
for msg in tqdm(train['clean_msg'], desc='going through...'):
  for word in msg.split(' '):
    word = word.lower()
    if word not in unique_words.keys():
      unique_words[word] = 1
    else:
      unique_words[word] += 1

print()
print(len(unique_words))

remove_email_words = set()
for word in unique_words.keys():
  if 'gmail' in word and 'com' in word:
    remove_email_words.add(word)

print(len(remove_email_words))

def clean_msg(msg):
  for word in msg.split(' '):
    if word in remove_email_words:
      msg = msg.replace(word,"")
  return msg

train['clean_msg1'] = train['clean_msg'].apply(clean_msg)

going through...: 100%|██████████| 85014/85014 [00:03<00:00, 26412.56it/s]



111429
3301


In [ ]:
train[['msg','clean_msg1']]

,msg,clean_msg1
0,I had continue received random calls and abusi...,continue received random calls abusive message...
1,The above fraudster is continuously messaging ...,fraudster continuously messaging asking pay mo...
2,He is acting like a police and demanding for m...,acting like police demanding money adding sect...
3,In apna Job I have applied for job interview f...,apna job applied job interview telecalling res...
4,I received a call from lady stating that she w...,received call lady stating send new phone vivo...
...,...,...
93681,Identity theft Smishing SMS Fraud CreditDeb...,identity theft smishing sms fraud creditdebit ...
93682,RECEIVED CALL FROM NUMBER ASKING ABOUT phone ...,received call number asking phone pay cash bac...
93683,Cyber Stalking Blackmailing PhoneSMSVOIP C...,cyber stalking blackmailing phonesmsvoip calls...
93684,Call karke bola ki aapka lotary laga ha aru AC...,call karke bola ki aapka lotary laga ha aru ac...


In [ ]:
unique_words1 = dict()
for msg in tqdm(train['clean_msg1'], desc='going through...'):
  for word in msg.split(' '):
    word = word.lower()
    if word not in unique_words1.keys():
      unique_words1[word] = 1
    else:
      unique_words1[word] += 1

print()
print(len(unique_words1))

going through...: 100%|██████████| 85014/85014 [00:02<00:00, 34430.65it/s]


108128


In [ ]:
unique_words_series = pd.Series(unique_words1).reset_index()
unique_words_series.columns = ['word', 'count']
unique_words_series.sort_values('count', ascending=False).to_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/unique_words_after_one_step.csv",index=False)

In [ ]:
train.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/train_cleaned.csv",index=False)
test.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/analysis_files/test_cleaned.csv",index=False)

## Base model

- SLL + BOW, TFIDF, Word embeddings + classification

## Modified model 1

- SRLL + BOW, TFIDF, Word embeddings + classification

## Modified model 2

- more refined SRLL + BOW, TFIDF, Word embeddings + classification

# Cleaned pipeline for data processing

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/IndiaAI_hackathon/final_usable_files/final_train.csv",low_memory=False)
print(train.shape)

test = pd.read_csv("/content/drive/MyDrive/IndiaAI_hackathon/final_usable_files/final_test.csv",low_memory=False)
print(test.shape)

def do_correction(text):
    changing_words = {'hai': 'is', 'ki': 'of', 'se': 'from', 'received': 'receive', 'ke': 'of', 'mere': 'my', 'started': 'start', 'ka': 'of',
    'dont': 'do not', 'rupees': 'rs', 'ko': 'to', 'aur': 'and', 'kiya': 'did', 'mujhe': 'me', 'par': 'but', 'liye': 'for', 'ek': 'one',
    'utr': 'answer', 'karta': 'does', 'maine': 'i', 'kr': 'do', 'isnt': 'is not', 'ho': 'yes', 'bhi': 'too', 'fir': 'then', 'mera': 'my', 'cant': 'can not',
    'aaya': 'came', 'paise': 'money', 'bola': 'said', 'diya': 'gave', 'im': 'i am', 'nhi': 'no', 'froud': 'fraud', 'ive': 'i have', 'nahi': 'no', 'liya': 'took',
    'ye': 'these', 'meri': 'my', 'plz': 'please', 'aap': 'you', 'gya': 'went', 'mene': 'i have', 'rupee': 'rs', 'hu': 'am', 'uske': 'his', 'jo': 'who', 'pls': 'please',
    'aadhar': 'aadhaar', 'karne': 'to do', 'gaya': 'went', 'wasnt': 'was not', 'inr': 'rs', 'wo': 'they', 'baad': 'after', 'rahe': 'are', 'de': 'give',
    'u': 'you', 'koi': 'any', 'paisa': 'rs', 'hua': 'happened', 'january': 'jan', 'sath': 'with', 'anydesk': 'any desk', 'nd': 'and', 'baroda': 'bob', 'frauder': 'fraud',
    'pese': 'rs', 'thats': 'that is', 'aya': 'came', 'kuch': 'some', 'krta': 'does', 'gye': 'went', 'karke': 'by doing', 'rhe': 'are', 'unhone': 'they', 'kaha': 'said',
    'na': 'no', 'bataya': 'told', 'fb': 'facebook', 'bol': 'tell', 'naam': 'names', 'krne': 'to do', 'unke': 'their', 'ab': 'now', 'phir': 'then', 'thi': 'was',
    'baat': 'talk', 'log': 'people', 'ref': 'reference', 'muje': 'me', 'oneoff': 'one off', 'rupey': 'rs', 'hadnt': 'had not', 'phonepay': 'phonepe', 'din': 'day',
    'everyones': 'everyone', 'relationships': 'relationship', 'diye': 'given', 'runaround': 'run around', 'kisi': 'any', 'vo': 'they', 'rupay': 'rs',
    'apne': 'our', 'ltd': 'limited', 'btaya': 'told', 'le': 'take', 'uska': 'his', 'aa': 'come', 'gaye': 'went', 'frod': 'fraud', 'february': 'feb', 'bhej': 'send',
    'hain': 'are', 'mein': 'in', 'dhamki': 'threat', 'rupe': 'rs', 'frouder': 'fraud', 'kare': 'do', 'bahut': 'very', 'rupaye': 'rs', 'december': 'dec',
    'apna': 'own', 'jab': 'when', 'ie': 'that is', 'iam': 'i am', 'sirmadam': 'sir madam', 'lekin': 'but', 'dene': 'to give', 'messaged': 'messaged',
    'abhi': 'now', 'mujhse': 'from me', 'aapko': 'to you', 'vedio': 'video', 'bat': 'talk', 'pas': 'near', 'ammount': 'amount', 'creditdebit': 'credit debit',
    'ha': 'yes', 'lo': 'take', 'toh': 'so', 'karna': 'to do', 'ur': 'your', 'didnot': 'did not', 'sab': 'all', 'rahi': 'doing', 'karo': 'do it', 'gali': 'street',
    'inke': 'their', 'pvt': 'private', 'holdreverse': 'hold reverse', 'jis': 'that', 'hoga': 'will be', 'kiye': 'did', 'dena': 'give', 'apko': 'to you',
    'watsapp': 'whatsapp', 'krke': 'by doing', 'nikal': 'remove', 'asap': 'as soon as possible', 'karte': 'do', 'usko': 'for him', 'suspecious': 'suspicious',
    'tak': 'till', 'jayega': 'will go', 'bheja': 'sent', 'peoples': 'people', 'unathorised': 'unauthorised', 'otps': 'otp', 'hackd': 'hacked', 'ap': 'you',
    'jisme': 'in which', 'hrs': 'hours', 'dwara': 'by', 'laga': 'put', 'ji': 'yes', 'fruad': 'fraud', 'bta': 'tell me', 'jaye': 'go', 'aapke': 'your',
    'aaj': 'today', 'pata': 'know', 'kijiye': 'do it', 'ni': 'no', 'jiska': 'whose', 'pornographic': 'pornography', 'fruader': 'fraud', 'fruadster': 'fraud',
    'fruads': 'fraud', 'fruaded': 'fraud', 'gpayphone': 'gpay phone', 'phonepegpay': 'phonepe gpay', 'fruade': 'fraud', 'fruadent': 'fraud', 'paytmgpay': 'paytm gpay',
    'gpayphonepay': 'gpay phonepe', 'pornograpy': 'pornography', 'fruadest': 'fraud', 'fruadsters': 'fraud', 'fruading': 'fraud', 'cimplaint': 'complaint',
    'gpaypaytm': 'gpay paytm', 'gpayeuronettotal': 'gpay euronettotal', 'phonepaygpay': 'phonepe gpay', 'pornogragy': 'pornography', 'childporns': 'child porn',
    'gpayat': 'gpay at', 'gpayhe': 'is gpay', 'govtsalary': 'government salary', 'gpaymy': 'gpay my', 'gpayphonepe': 'gpay phonepe', 'bygpay': 'by gpay',
    'gpayed': 'gpay', 'gpaybut': 'gpay but', 'gpaythey': 'gpay they', 'afruad': 'a fraud', 'fruadulant': 'fraud', 'pornograpgy': 'pornography',
    'gpaythen': 'gpay then', 'fruadulent': 'fraud', 'hellomy': 'hello my', 'bankingpay': 'banking pay', 'gpayinitial': 'gpay initial', 'gpaylater': 'gpay later',
    'whatappgpay': 'whatsapp gpay', 'fruadamount': 'fraud amount', 'lostvictim': 'lost victim', 'padhega': 'will read', 'varificaon': 'verification',
    'maiupulating': 'manipulating', 'nowadys': 'now a days', 'hasband': 'husband', 'subcribers': 'subscriber', 'youtubewhen': 'youtube when', 'instarction': 'interaction',
    'cardupimobile': 'card upi mobile', 'rejecied': 'rejected', 'bhahen': 'sister', 'comingthey': 'coming they', 'amountfirsy': 'amount first', 'uhave': 'you have',
    'roadowner': 'road owner', 'readable': 'read able', 'paasport': 'passport', 'latere': 'later', 'tryinh': 'trying', 'gpayon': 'gpay on', 'praveengpay': 'praveen gpay',
    'upipaytmgpay': 'upi paytm gpay', 'gpaylink': 'gpay link', 'gpayamount': 'gpay amount', 'intrnet': "internet", "secreate": "secret", "victime": "victim",
    "apka": "your", "apke": "your", 'aapka': 'yours', 'abusive': 'abuse', 'acc': 'account', 'accidentally': 'accident', 'ad': 'advertisement', 'adhar': 'aadhaar', 'alag': 'apart',
    'amt': 'amount', 'apka': 'your', 'apke': 'your', 'app': 'application', 'approval': 'approve', 'attachment': 'attach', 'baar': 'times', 'band': 'close',
    'bankar': 'become', 'banking': 'bank', 'bar': 'times', 'bata': 'tell', 'blackmailing': 'blackmail', 'booking': 'book', 'careful': 'care',
    'chala': 'walked', 'chesaru': 'done', 'clothe': 'cloth', 'complainant': 'complaint', 'completely': 'complete', 'confirmation': 'confirm',
    'constantly': 'constant', 'continuously': 'continuous', 'dal': 'did', 'deduction': 'deduct', 'delivery': 'deliver', 'denge': 'give',
    'dhani': 'rich', 'di': 'gave', 'dijiye': 'give', 'directly': 'direct', 'dist': 'district', 'dunga': 'give', 'firstly': 'first', 'fraudster': 'fraud',
    'fraudulent': 'fraud', 'frezzing': 'freeze', 'froder': 'fraud', 'frustrated': 'frustrate', 'galat': 'wrong', 'gandi': 'dirty', 'getting': 'get',
    'googlepay': 'gpay', 'harassment': 'harass', 'harrasse': 'harass', 'holder': 'hold', 'hone': 'to', 'honestly': 'honest', 'hum': 'we',
    'immediately': 'immediate', 'impersonation': 'impersonate', 'informed': 'inform', 'initially': 'initial', 'insta': 'instagram', 'instal': 'install',
    'instruction': 'instruct', 'investigation': 'investigate', 'isne': 'has', 'ja': 'go', 'jankar': 'knowing', 'jankari': 'information',
    'jiske': 'whose', 'kar': 'do', 'karwa': 'did', 'kat': 'cut', 'kra': 'do', 'kro': 'do', 'kya': 'what', 'lac': 'lakh', 'ladki': 'lady', 'lal': 'red',
    'li': 'took', 'log': 'people', 'lone': 'loan', 'mai': 'i', 'mang': 'demand', 'mentally': 'mental', 'mila': 'found', 'min': 'minimum', 'mo': 'mobile',
    'mob': 'mobile', 'msg': 'message', 'msgs': 'message', 'nam': 'name', 'ne': 'near', 'nos': 'number', 'notifiction': 'notification', 'num': 'number',
    'pahle': 'first', 'pareshan': 'worried', 'pe': 'pay', 'pende': 'pending', 'ph': 'phone', 'pic': 'picture', 'pr': 'but', 'pta': 'know', 'raha': 'remained',
    'recieve': 'receive', 'recive': 'receive', 'recording': 'record', 'reference': 'reference', 'refrence': 'reference', 'refunded': 'refund', 'rha': 'remained',
    'rhi': 'doing', 'sare': 'ok', 'saving': 'save', 'scamme': 'scammer', 'smishe': 'funny', 'st': 'street', 'stange': 'strange', 'suchana': 'information',
    'suchna': 'information', 'suchnakrta': 'inform ', 'terrifying': 'terrify', 'texte': 'text', 'thana': 'station', 'threate': 'threat', 'trans': 'transaction',
    'transacation': 'transaction', 'transaction': 'transaction', 'transction': 'transaction', 'transection': 'transaction', 'usme': 'in that', 'usne': 'he',
    'vishe': 'special', 'wale': 'ones', 'wapas': 'back', 'week': 'weak', 'whatsup': 'whatsapp', 'withdrawal': 'withdraw', 'working': 'work', 'ya': 'yes'
    }

    # Correct the words
    corrected_msg = ''
    all_keys = changing_words.keys()
    for word in text.split():
        if word in all_keys:
            corrected_msg += changing_words[word]
        else:
            corrected_msg += word
        corrected_msg += " "
    return corrected_msg

def do_sensitive_website_replacement(corrected_msg, website_match_findings):
  sensitive_keywords = [
        "porn", "xxx", "hentai", "xvideo", "adult", "breast","pussy", "dick", "cock","sex", "erotic", "nude", "camgirl", "cams", "cheat",
        "escort", "fetish", "bdsm", "taboo", "incest", "shemale", "milf", "barelylegal", "amateur",
        "anal", "fetish", "hotgirls", "kinky", "hardcore", "lingerie", "webcam", "strip", "nsfw",
        "onlyfans", "redtube", "youjizz", "spankbang", "tnaflix", "xtube", "nudity", "playboy",
        "bangbros", "brazzers", "naughty", "mature", "teens", "voyeur", "threesome", "gangbang",
        "peeping", "exposed", "babes", "slut", "masturbate", "hookup", "sugarbaby", "paypig", "comic"
  ]

  new_msg = ''
  for word in corrected_msg.strip().split():
    # print(word)
    if len(website_match_findings) > 0:
        if word in website_match_findings and 'express' not in word and 'gmail' not in word and 'internet' not in word:
            # Check if it has sensitive word
            for sen_word in sensitive_keywords:
                if sen_word in word:
                    new_msg += sen_word
        else:
          new_msg += word
    else:
        new_msg += word
    new_msg += " "
  return new_msg

(84040, 16)
(28328, 15)


## Stopwords

In [ ]:
stop_words = [
    "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according",
    "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag",
    "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although",
    "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone",
    "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar",
    "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available",
    "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes",
    "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside",
    "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief",
    "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause",
    "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co",
    "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding",
    "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d",
    "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different",
    "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due",
    "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el",
    "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et",
    "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey",
    "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo",
    "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full",
    "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes",
    "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt",
    "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein",
    "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully",
    "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id",
    "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch",
    "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward",
    "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr",
    "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last",
    "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely",
    "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make",
    "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million",
    "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't",
    "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't",
    "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless",
    "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained",
    "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only",
    "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over",
    "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas",
    "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible",
    "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud",
    "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re",
    "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research",
    "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt",
    "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section",
    "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf",
    "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown",
    "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm",
    "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp",
    "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such",
    "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten",
    "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves",
    "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto",
    "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this",
    "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til",
    "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying",
    "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately",
    "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses",
    "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs",
    "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went",
    "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's",
    "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither",
    "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish",
    "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't",
    "xt", "xv", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself",
    "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz", 'ag', 'amit', 'ani', 'dec', 'delhi', 'devi', 'feb', 'five', 'gupta', 'hi', 'hui',
    'jaipur', 'jan', 'kumar', 'may', 'rahul', 'ram', 'sharma', 'singh', 'yadav', 'mar', 'apr', 'jun', 'jul', 'aug', 'sept', 'oct', 'nov']

## Pipeline

In [ ]:
# final SLL msg
from tqdm import tqdm
## DATA PROCESSING
'''
1. stopwords
2. lower
3. replace mispelled words
4. detect websites and detect sensitive words and replace them, else remove
5. lemma
'''

'\n1. stopwords\n2. lower\n3. replace mispelled words\n4. detect websites and detect sensitive words and replace them, else remove\n5. lemma\n'

In [ ]:
train_SLL = []
train_SRLL = []
train_SRULL = []

for msg in tqdm(train['msg'], desc = 'processing train:'):
  # SLL
  text = msg.lower().strip()
  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words]
  # Lemmatization using spacy
  doc_SLL = nlp(' '.join(tokens))
  lemmatized_tokens_SLL = [token.lemma_ for token in doc_SLL]
  train_SLL.append(' '.join(lemmatized_tokens_SLL).strip())

  # SRLL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)
  tokens = corrected_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  doc_SRLL = nlp(' '.join(tokens))
  lemmatized_tokens_SRLL = [token.lemma_ for token in doc_SRLL]
  train_SRLL.append(' '.join(lemmatized_tokens_SRLL).strip())

  # SRULL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)

  doubt_website_pattern = r'\b(?:http|https|www)[a-zA-Z0-9]+(?:dot|at)?[a-zA-Z0-9]+(?:com|org|net)\b|\b[a-zA-Z0-9._%+-]+(?:at|dot)?[a-zA-Z0-9]+(?:com|org|net)\b'
  website_match_findings = re.findall(doubt_website_pattern, corrected_msg)
  new_msg = do_sensitive_website_replacement(corrected_msg, website_match_findings)

  tokens = new_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  doc_SRULL = nlp(' '.join(tokens))
  lemmatized_tokens_SRULL = [token.lemma_ for token in doc_SRULL]
  train_SRULL.append(' '.join(lemmatized_tokens_SRULL).strip())

train['SLL_msg_updated'] = train_SLL
train['SRLL_msg_updated'] = train_SRLL
train['SRULL_msg_updated'] = train_SRULL

train.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/final_usable_files/final_train.csv",index=False)
print("Train saved")

processing train:: 100%|██████████| 84040/84040 [55:34<00:00, 25.21it/s]


Train saved


In [ ]:
test_SLL = []
test_SRLL = []
test_SRULL = []

for msg in tqdm(test['msg'], desc = 'processing test:'):
  # SLL
  text = msg.lower().strip()
  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words]
  # Lemmatization using spacy
  doc_SLL = nlp(' '.join(tokens))
  lemmatized_tokens = [token.lemma_ for token in doc_SLL]
  test_SLL.append(' '.join(lemmatized_tokens).strip())

  # SRLL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)
  tokens = corrected_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  doc_SRLL = nlp(' '.join(tokens))
  lemmatized_tokens = [token.lemma_ for token in doc_SRLL]
  test_SRLL.append(' '.join(lemmatized_tokens).strip())

  # SRULL
  text = msg.lower().strip()
  corrected_msg = do_correction(text)

  doubt_website_pattern = r'\b(?:http|https|www)[a-zA-Z0-9]+(?:dot|at)?[a-zA-Z0-9]+(?:com|org|net)\b|\b[a-zA-Z0-9._%+-]+(?:at|dot)?[a-zA-Z0-9]+(?:com|org|net)\b'
  website_match_findings = re.findall(doubt_website_pattern, corrected_msg)
  new_msg = do_sensitive_website_replacement(corrected_msg, website_match_findings)

  tokens = new_msg.split()
  tokens = [word for word in tokens if word not in stop_words]
  doc_SRULL = nlp(' '.join(tokens))
  lemmatized_tokens = [token.lemma_ for token in doc_SRULL]
  test_SRULL.append(' '.join(lemmatized_tokens).strip())

test['SLL_msg_updated'] = test_SLL
test['SRLL_msg_updated'] = test_SRLL
test['SRULL_msg_updated'] = test_SRULL

test.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/final_usable_files/final_test.csv",index=False)

processing test:: 100%|██████████| 28328/28328 [21:53<00:00, 21.56it/s]


In [ ]:
test.columns

Index(['msg', 'SRLL_bert_embeddings', 'category', 'sub_category', 'SLL_msg',
       'clean_msg_before_srll', 'SRLL_msg', 'final_category', 'before_SLL_msg',
       'SRLL_msg_before', 'SRULL_msg', 'SLL_msg_updated', 'SRLL_msg_updated',
       'SRULL_msg_updated', 'SLL_bert_embeddings'],
      dtype='object')

In [ ]:
test = test[['msg', 'category', 'sub_category', 'SLL_msg',
       'clean_msg_before_srll', 'SRLL_msg', 'final_category', 'before_SLL_msg',
       'SRLL_msg_before', 'SRULL_msg', 'SLL_msg_updated', 'SRLL_msg_updated',
       'SRULL_msg_updated']].reset_index(drop=True)

train = train[['msg', 'category', 'sub_category', 'SLL_msg',
       'clean_msg_before_srll', 'SRLL_msg', 'final_category', 'before_SLL_msg',
       'SRLL_msg_before', 'SRULL_msg', 'SLL_msg_updated', 'SRLL_msg_updated',
       'SRULL_msg_updated']].reset_index(drop=True)

In [ ]:
train.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/final_usable_files/final_train.csv",index=False)
test.to_csv("/content/drive/MyDrive/IndiaAI_hackathon/final_usable_files/final_test.csv", index=False)

In [ ]:
test.head()

,msg,category,sub_category,SLL_msg,clean_msg_before_srll,SRLL_msg,final_category,before_SLL_msg,SRLL_msg_before,SRULL_msg,SLL_msg_updated,SRLL_msg_updated,SRULL_msg_updated
0,Sir namaskar mein Ranjit Kumar PatraPaise neh...,RapeGang Rape RGRSexually Abusive Content,NaN,sir namaskar mein ranjit kumar patrapaise nehi...,sir namaskar mein ranjit kumar patrapaise nehi...,sir namaskar ranjit kumar patrapaise nehi tho ...,0,sir namaskar mein ranjit kumar patrapaise neh...,sir namaskar in ranjit kumar patrapaise nehi t...,sir namaskar ranjit kumar patrapaise nehi tho ...,sir namaskar mein ranjit patrapaise nehi tho s...,sir namaskar ranjit patrapaise nehi tho sir da...,sir namaskar ranjit patrapaise nehi tho sir da...
1,KOTAK MAHINDRA BANK FRAUD\r\nFRAUD AMOUNT,Online Financial Fraud,DebitCredit Card FraudSim Swap Fraud,kotak mahindra bank fraud fraud amount,kotak mahindra bank fraud fraud amount,kotak mahindra bank fraud fraud amount,1,kotak mahindra bank fraud\r\nfraud amount,kotak mahindra bank fraud fraud amount,kotak mahindra bank fraud fraud amount,kotak mahindra bank fraud fraud,kotak mahindra bank fraud fraud,kotak mahindra bank fraud fraud
2,The issue actually started when I got this ema...,Cyber Attack/ Dependent Crimes,SQL Injection,issue actually start get email first glance se...,issue actually started got email first glance ...,issue actually start get email first glance se...,2,the issue actually started when i got this ema...,the issue actually start when i got this email...,"issue actually start get email , first glance ...","issue start email , glance spam . ignore thing...","issue start email , glance spam . ignore thing...","issue start email , glance spam . ignore thing..."
3,I am amit kumar from karwi chitrakoot I am tot...,Online Financial Fraud,Fraud CallVishing,amit kumar karwi chitrakoot totally depress fr...,amit kumar karwi chitrakoot totally depressed ...,amit kumar karwi chitrakoot totally depress fr...,1,i am amit kumar from karwi chitrakoot i am tot...,i am amit kumar from karwi chitrakoot i am tot...,amit kumar karwi chitrakoot totally depress fr...,karwi chitrakoot totally depress fraud call ms...,karwi chitrakoot totally depress fraud call me...,karwi chitrakoot totally depress fraud call me...
4,I have ordered saree and blouse from rinki s...,Any Other Cyber Crime,Other,order saree blouse rinki sur pay amount take p...,ordered saree blouse rinki sur paid amount tak...,order saree blouse rinki sur pay amount take p...,2,i have ordered saree and blouse from rinki s...,i have ordered saree and blouse from rinki sur...,order saree blouse rinki sur pay amount take p...,order saree blouse rinki sur pay payment do no...,order saree blouse rinki sur pay payment do no...,order saree blouse rinki sur pay payment do no...
